In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import nltk
import re
import math

In [3]:
key = pd.read_csv('key_english.csv')

In [4]:
cr = pd.read_csv('crossrefs.txt',sep=' ')

In [5]:
cr.head()

,From,Verse
0,Gen.1.1,Isa.65.17
1,Gen.1.1,Ps.33.6
2,Gen.1.1,Job.38.4
3,Gen.1.1,Rev.22.13
4,Gen.1.1,Rev.10.6


In [6]:
key.head()

,field,field.1,field.2,field.3
0,1,Genesis,OT,1
1,2,Exodus,OT,1
2,3,Leviticus,OT,1
3,4,Numbers,OT,1
4,5,Deuteronomy,OT,1


In [7]:
#cr['fromch'] = cr['From'].replace(to_replace="(\d|\.)", value="", regex=True)
#cr['versech'] = cr['Verse'].replace(to_replace="(\d|\.)", value="", regex=True)
cr['frombk'] = cr['From'].str.extract(r'(\d?\w+)',expand=False)
cr['versebk'] = cr['Verse'].str.extract(r'(\d?\w+)',expand=False)

In [8]:
cr['fnums'] = cr['From'].str.extract(r'(\.\d{1,3}\.\d{1,3})',expand=False)
cr['vnums'] = cr['Verse'].str.extract(r'(\.\d{1,3}\.\d{1,3})',expand=False)


In [9]:
#cr['From'].apply(lambda x: x.split('.'))

In [10]:
cr.head(20)

,From,Verse,frombk,versebk,fnums,vnums
0,Gen.1.1,Isa.65.17,Gen,Isa,.1.1,.65.17
1,Gen.1.1,Ps.33.6,Gen,Ps,.1.1,.33.6
2,Gen.1.1,Job.38.4,Gen,Job,.1.1,.38.4
3,Gen.1.1,Rev.22.13,Gen,Rev,.1.1,.22.13
4,Gen.1.1,Rev.10.6,Gen,Rev,.1.1,.10.6
5,Gen.1.1,Ps.136.5,Gen,Ps,.1.1,.136.5
6,Gen.1.1,Acts.17.24,Gen,Acts,.1.1,.17.24
7,Gen.1.1,Isa.45.18,Gen,Isa,.1.1,.45.18
8,Gen.1.1,Acts.14.15,Gen,Acts,.1.1,.14.15
9,Gen.1.1,Rev.14.7,Gen,Rev,.1.1,.14.7


In [11]:
abvlist = pd.unique(cr['frombk'])

In [12]:
idlist = [i for i,el in enumerate(pd.unique(cr['versebk']))]

In [13]:
abvlist

array(['Gen', 'Exod', 'Lev', 'Num', 'Deut', 'Josh', 'Judg', 'Ruth',
       '1Sam', '2Sam', '1Kgs', '2Kgs', '1Chr', '2Chr', 'Ezra', 'Neh',
       'Esth', 'Job', 'Ps', 'Prov', 'Eccl', 'Song', 'Isa', 'Jer', 'Lam',
       'Ezek', 'Dan', 'Hos', 'Joel', 'Amos', 'Obad', 'Jonah', 'Mic',
       'Nah', 'Hab', 'Zeph', 'Hag', 'Zech', 'Mal', 'Matt', 'Mark', 'Luke',
       'John', 'Acts', 'Rom', '1Cor', '2Cor', 'Gal', 'Eph', 'Phil', 'Col',
       '1Thess', '2Thess', '1Tim', '2Tim', 'Titus', 'Phlm', 'Heb', 'Jas',
       '1Pet', '2Pet', '1John', '2John', '3John', 'Jude', 'Rev'],
      dtype=object)

In [14]:
key['abv'] = [i for i in abvlist]

In [15]:
key.head()

,field,field.1,field.2,field.3,abv
0,1,Genesis,OT,1,Gen
1,2,Exodus,OT,1,Exod
2,3,Leviticus,OT,1,Lev
3,4,Numbers,OT,1,Num
4,5,Deuteronomy,OT,1,Deut


In [16]:
def match_abv(d):
    l = []
    #for j in cr['versech']:
    for j in d:
        for i,el in enumerate(key['abv']):
            if el == j:
                l.append(i+1)
    return l

In [17]:
cr['v_bk_id']= match_abv(cr['versebk'])

In [18]:
cr['f_bk_id']= match_abv(cr['frombk'])

In [19]:
cr.head()

,From,Verse,frombk,versebk,fnums,vnums,v_bk_id,f_bk_id
0,Gen.1.1,Isa.65.17,Gen,Isa,.1.1,.65.17,23,1
1,Gen.1.1,Ps.33.6,Gen,Ps,.1.1,.33.6,19,1
2,Gen.1.1,Job.38.4,Gen,Job,.1.1,.38.4,18,1
3,Gen.1.1,Rev.22.13,Gen,Rev,.1.1,.22.13,66,1
4,Gen.1.1,Rev.10.6,Gen,Rev,.1.1,.10.6,66,1


In [20]:
df = pd.read_csv('t_asv.csv')

In [21]:
df['id'] = df['field'].apply('{:>08d}'.format)

In [22]:
df.head()

,field,field.1,field.2,field.3,field.4,id
0,1001001,1,1,1,In the beginning God created the heavens and t...,01001001
1,1001002,1,1,2,And the earth was waste and void; and darkness...,01001002
2,1001003,1,1,3,"And God said, Let there be light: and there wa...",01001003
3,1001004,1,1,4,"And God saw the light, that it was good: and G...",01001004
4,1001005,1,1,5,"And God called the light Day, and the darkness...",01001005


In [23]:
vchnum = cr['vnums'].apply(lambda x: x.split('.'))
fchnum = cr['fnums'].apply(lambda x: x.split('.'))

In [24]:
for i,el in enumerate(vchnum):
    el[0] = cr['v_bk_id'][i]


In [25]:
for i,el in enumerate(fchnum):
    el[0] = cr['f_bk_id'][i]

In [26]:
for i,el in enumerate(vchnum):
    for j,jel in enumerate(el):
        vchnum[i][j] = '{:>03d}'.format(int(jel))
        

In [27]:
for i,el in enumerate(fchnum):
    for j,jel in enumerate(el):
        fchnum[i][j] = '{:>03d}'.format(int(jel))

In [28]:
vchnum

0         [023, 065, 017]
1         [019, 033, 006]
2         [018, 038, 004]
3         [066, 022, 013]
4         [066, 010, 006]
5         [019, 136, 005]
6         [044, 017, 024]
7         [023, 045, 018]
8         [044, 014, 015]
9         [066, 014, 007]
10        [061, 003, 005]
11        [021, 012, 001]
12        [019, 104, 024]
13        [058, 003, 004]
14        [020, 008, 022]
15        [040, 011, 025]
16        [041, 013, 019]
17        [046, 008, 006]
18        [019, 115, 015]
19        [019, 121, 002]
20        [024, 032, 017]
21        [020, 003, 019]
22        [019, 104, 030]
23        [019, 134, 003]
24        [019, 146, 006]
25        [002, 020, 011]
26        [024, 051, 015]
27        [043, 001, 001]
28        [023, 051, 016]
29        [020, 016, 004]
               ...       
343579    [066, 002, 007]
343580    [042, 011, 052]
343581    [066, 003, 004]
343582    [066, 022, 002]
343583    [066, 021, 022]
343584    [005, 004, 002]
343585    [002, 032, 033]
343586    [0

In [29]:
vchnum = [int((''.join(map(str,x)))) for x in vchnum]

In [30]:
fchnum = [int((''.join(map(str,x)))) for x in fchnum]

In [31]:
fchnum

[1001001,
 1001001,
 1001001,
 1001001,
 1001001,
 1001001,
 1001001,
 1001001,
 1001001,
 1001001,
 1001001,
 1001001,
 1001001,
 1001001,
 1001001,
 1001001,
 1001001,
 1001001,
 1001001,
 1001001,
 1001001,
 1001001,
 1001001,
 1001001,
 1001001,
 1001001,
 1001001,
 1001001,
 1001001,
 1001001,
 1001001,
 1001001,
 1001001,
 1001001,
 1001001,
 1001001,
 1001001,
 1001001,
 1001001,
 1001001,
 1001001,
 1001001,
 1001001,
 1001001,
 1001001,
 1001001,
 1001001,
 1001001,
 1001001,
 1001001,
 1001001,
 1001001,
 1001001,
 1001001,
 1001001,
 1001001,
 1001001,
 1001001,
 1001001,
 1001001,
 1001001,
 1001002,
 1001002,
 1001002,
 1001003,
 1001003,
 1001003,
 1001003,
 1001003,
 1001003,
 1001003,
 1001003,
 1001003,
 1001003,
 1001003,
 1001003,
 1001003,
 1001003,
 1001003,
 1001003,
 1001003,
 1001003,
 1001004,
 1001004,
 1001004,
 1001004,
 1001004,
 1001004,
 1001004,
 1001005,
 1001005,
 1001005,
 1001005,
 1001005,
 1001005,
 1001005,
 1001005,
 1001005,
 1001005,
 1001005,


In [32]:
cr['v_ids'] = vchnum
cr['f_ids'] = fchnum

In [33]:
cr.head()

,From,Verse,frombk,versebk,fnums,vnums,v_bk_id,f_bk_id,v_ids,f_ids
0,Gen.1.1,Isa.65.17,Gen,Isa,.1.1,.65.17,23,1,23065017,1001001
1,Gen.1.1,Ps.33.6,Gen,Ps,.1.1,.33.6,19,1,19033006,1001001
2,Gen.1.1,Job.38.4,Gen,Job,.1.1,.38.4,18,1,18038004,1001001
3,Gen.1.1,Rev.22.13,Gen,Rev,.1.1,.22.13,66,1,66022013,1001001
4,Gen.1.1,Rev.10.6,Gen,Rev,.1.1,.10.6,66,1,66010006,1001001


In [34]:
cr['v_ids'] = cr['v_ids'].apply('{:>08d}'.format)
cr['f_ids'] = cr['f_ids'].apply('{:>08d}'.format)

In [35]:
cr.head(40)

,From,Verse,frombk,versebk,fnums,vnums,v_bk_id,f_bk_id,v_ids,f_ids
0,Gen.1.1,Isa.65.17,Gen,Isa,.1.1,.65.17,23,1,23065017,01001001
1,Gen.1.1,Ps.33.6,Gen,Ps,.1.1,.33.6,19,1,19033006,01001001
2,Gen.1.1,Job.38.4,Gen,Job,.1.1,.38.4,18,1,18038004,01001001
3,Gen.1.1,Rev.22.13,Gen,Rev,.1.1,.22.13,66,1,66022013,01001001
4,Gen.1.1,Rev.10.6,Gen,Rev,.1.1,.10.6,66,1,66010006,01001001
5,Gen.1.1,Ps.136.5,Gen,Ps,.1.1,.136.5,19,1,19136005,01001001
6,Gen.1.1,Acts.17.24,Gen,Acts,.1.1,.17.24,44,1,44017024,01001001
7,Gen.1.1,Isa.45.18,Gen,Isa,.1.1,.45.18,23,1,23045018,01001001
8,Gen.1.1,Acts.14.15,Gen,Acts,.1.1,.14.15,44,1,44014015,01001001
9,Gen.1.1,Rev.14.7,Gen,Rev,.1.1,.14.7,66,1,66014007,01001001


In [36]:
toex = pd.DataFrame(cr['f_ids'])

In [37]:
toex['v_ids'] = cr['v_ids']

In [38]:
toex

,f_ids,v_ids
0,01001001,23065017
1,01001001,19033006
2,01001001,18038004
3,01001001,66022013
4,01001001,66010006
5,01001001,19136005
6,01001001,44017024
7,01001001,23045018
8,01001001,44014015
9,01001001,66014007


In [39]:
toex.to_csv('crossref_ids.tsv',sep='\t')

In [49]:
gp = toex.groupby('f_ids')['v_ids'].apply(list)

In [51]:
gp.to_json('cr_ids.json')